# Data_Clean_computrabajo

In [1]:
import pandas as pd 
import numpy as np
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from collections import Counter
from wordcloud import STOPWORDS
import spacy
# Descargar stopwords si no las tienes
nltk.download('stopwords')

# Cargar modelo spaCy español
nlp = spacy.load('es_core_news_sm')
stopwords_es = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\John\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Cargaremos y eliminaremos las ofertas duplicadas, debido a que los postulantes suelen renovar estás ofertas, y por buenas prácticas dejaremos el URL cómo primary key (aunque no manejaremos datos cruzados).

In [2]:
df = pd.read_csv("../data/ofertas_computrabajo.csv")

# Normaliza nombres de columnas a minúsculas si fuera necesario
df.columns = df.columns.str.lower()

# Mover la columna 'url' al inicio
columnas = ["url"] + [col for col in df.columns if col != "url"]

# Eliminar duplicados
df = df.drop_duplicates(subset=["title", "company", "rating", "location", "salary", 
                                "contract", "schedule", "description", "education", 
                                "experience", "age", "skills"], keep="first")

df = df[columnas]

df.head(5)


,url,title,company,rating,location,salary,contract,schedule,description,education,experience,age,skills
0,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de programación y datos - Medellín,Empleamos Temporales SAS,"4,45","Medellín, Antioquia",A convenir,Contrato de Obra o labor,Tiempo Completo,¡Estamos buscando talento en programación y an...,Universidad / Carrera tecnológica,1 año de experiencia,No disponible,No disponible
1,https://co.computrabajo.com/ofertas-de-trabajo...,Analista Operaciones / Analista de datos,Corporación Interactuar,No disponible,"Bello, Antioquia",A convenir,Contrato a término indefinido,Tiempo Completo,Si eres un apasionado por el análisis de datos...,Universidad / Carrera Profesional,No disponible,No disponible,No disponible
2,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos / Analista BI // Manejo de i...,BRM S.A.S,"4,24","Medellín, Antioquia","$ 4.000.000,00 (Mensual)",Contrato a término indefinido,Tiempo Completo,"¿Eres un detective de datos nato, capaz de enc...",Universidad / Carrera Profesional,2 años de experiencia,A partir de 18 años,No disponible
3,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos,Alianza Temporal S.A.S,"4,31","Bogotá, D.C., Bogotá, D.C.","$ 2.000.000,00 (Mensual)",Contrato de Obra o labor,Tiempo Completo,Importante empresa del sector logistica y tran...,Universidad / Carrera Profesional,3 años de experiencia,No disponible,No disponible
4,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos / Experiencia en facturación...,BRM S.A.S,"4,24","Bogotá, D.C., Bogotá, D.C.","$ 2.500.000,00 (Mensual)",Contrato a término indefinido,Tiempo Completo,¿Eres un gurú de los datos financieros con una...,Universidad / Carrera técnica,1 año de experiencia,A partir de 18 años,No disponible


### Clean_description

In [3]:
nlp = spacy.load('es_core_news_sm')

def normalizar_y_lematizar(texto):
    if pd.isna(texto):
        return texto

    # Conservar signos útiles: $, : , . y números
    texto = re.sub(r"[^a-zA-Z0-9áéíóúÁÉÍÓÚñÑ\s$:.,]", "", texto)
    
    # Procesar con spaCy
    doc = nlp(texto)

    lemas = []
    for token in doc:
        if token.is_space:
            continue
        if token.is_stop and not token.like_num:
            continue
        lemas.append(token.lemma_)

    return " ".join(lemas)


In [4]:
df['description'] = df['description'].apply(normalizar_y_lematizar)

### Clean_rating

In [5]:
# Reemplazar comas por puntos y convertir a numérico
df["rating"] = df["rating"].astype(str).str.replace(",", ".", regex=False)
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df.head(5)

,url,title,company,rating,location,salary,contract,schedule,description,education,experience,age,skills
0,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de programación y datos - Medellín,Empleamos Temporales SAS,4.45,"Medellín, Antioquia",A convenir,Contrato de Obra o labor,Tiempo Completo,buscar talento programación análisis datosempl...,Universidad / Carrera tecnológica,1 año de experiencia,No disponible,No disponible
1,https://co.computrabajo.com/ofertas-de-trabajo...,Analista Operaciones / Analista de datos,Corporación Interactuar,NaN,"Bello, Antioquia",A convenir,Contrato a término indefinido,Tiempo Completo,"apasionado análisis dato , oferta corporación ...",Universidad / Carrera Profesional,No disponible,No disponible,No disponible
2,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos / Analista BI // Manejo de i...,BRM S.A.S,4.24,"Medellín, Antioquia","$ 4.000.000,00 (Mensual)",Contrato a término indefinido,Tiempo Completo,"detective dato nato , capaz encontrar aguja co...",Universidad / Carrera Profesional,2 años de experiencia,A partir de 18 años,No disponible
3,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos,Alianza Temporal S.A.S,4.31,"Bogotá, D.C., Bogotá, D.C.","$ 2.000.000,00 (Mensual)",Contrato de Obra o labor,Tiempo Completo,importante empresa sector logistico transporte...,Universidad / Carrera Profesional,3 años de experiencia,No disponible,No disponible
4,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos / Experiencia en facturación...,BRM S.A.S,4.24,"Bogotá, D.C., Bogotá, D.C.","$ 2.500.000,00 (Mensual)",Contrato a término indefinido,Tiempo Completo,gurú dato financiero obsesión exactitud factur...,Universidad / Carrera técnica,1 año de experiencia,A partir de 18 años,No disponible


## Clean Location

In [6]:
# Eliminar espacios en blanco alrededor de los valores 
df["location"] = df["location"].str.strip()

# Separar la columna en "city" y "department" usando la última coma como separador
df[["city", "department"]] = df["location"].str.rsplit(", ", n=1, expand=True)

# Manejar caso especial de "Bogotá, D.C., Bogotá, D.C."
df.loc[df["location"] == "Bogotá, D.C., Bogotá, D.C.", ["city", "department"]] = ["Bogotá, D.C.", "Bogotá, D.C."]

# Si department es nulo o vacío, asignarle el valor de city
df["department"] = df["department"].fillna(df["city"])  # Para valores NaN
df.loc[df["department"].str.strip() == "", "department"] = df["city"]  # Para valores vacíos
df['city'] = df['city'].replace({
    'San Andres, Archipíelago De San Andrés': 'San Andrés',
    'Siberia': 'Cota' 
})


# Eliminar la columna location
df.drop(columns=["location"], inplace=True)

# Verificar resultado
df.head(3)


,url,title,company,rating,salary,contract,schedule,description,education,experience,age,skills,city,department
0,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de programación y datos - Medellín,Empleamos Temporales SAS,4.45,A convenir,Contrato de Obra o labor,Tiempo Completo,buscar talento programación análisis datosempl...,Universidad / Carrera tecnológica,1 año de experiencia,No disponible,No disponible,Medellín,Antioquia
1,https://co.computrabajo.com/ofertas-de-trabajo...,Analista Operaciones / Analista de datos,Corporación Interactuar,NaN,A convenir,Contrato a término indefinido,Tiempo Completo,"apasionado análisis dato , oferta corporación ...",Universidad / Carrera Profesional,No disponible,No disponible,No disponible,Bello,Antioquia
2,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos / Analista BI // Manejo de i...,BRM S.A.S,4.24,"$ 4.000.000,00 (Mensual)",Contrato a término indefinido,Tiempo Completo,"detective dato nato , capaz encontrar aguja co...",Universidad / Carrera Profesional,2 años de experiencia,A partir de 18 años,No disponible,Medellín,Antioquia


## Clean Salary

In [7]:
conteo_ciudades = df['salary'].value_counts().sort_values(ascending=False).reset_index()
conteo_ciudades.columns = ['salary', 'frecuencia']
pd.set_option('display.max_rows', None)  # Muestra todas las filas
print(conteo_ciudades)

                        salary  frecuencia
0                   A convenir         546
1     $ 1.423.500,00 (Mensual)         148
2     $ 2.000.000,00 (Mensual)          84
3     $ 1.800.000,00 (Mensual)          59
4     $ 2.500.000,00 (Mensual)          52
5     $ 1.500.000,00 (Mensual)          52
6     $ 3.000.000,00 (Mensual)          50
7     $ 4.000.000,00 (Mensual)          25
8     $ 3.500.000,00 (Mensual)          24
9     $ 2.200.000,00 (Mensual)          22
10    $ 1.600.000,00 (Mensual)          21
11    $ 1.300.000,00 (Mensual)          16
12    $ 1.700.000,00 (Mensual)          16
13    $ 2.300.000,00 (Mensual)          16
14    $ 2.600.000,00 (Mensual)          15
15    $ 1.900.000,00 (Mensual)          13
16    $ 2.800.000,00 (Mensual)          13
17    $ 1.423.000,00 (Mensual)          10
18    $ 2.700.000,00 (Mensual)           9
19    $ 5.000.000,00 (Mensual)           9
20    $ 3.300.000,00 (Mensual)           9
21    $ 3.200.000,00 (Mensual)           9
22    $ 2.1

In [ ]:
def limpiar_salario(s):
    if pd.isna(s) or not isinstance(s, str):
        return None
    if 'a convenir' in s.lower():
        return None
    s = s.split('(')[0]  # eliminar "(Mensual)", "(Anual)", etc.
    s = s.replace(',00', '')  # eliminar decimales irrelevantes
    s = s.replace('$', '').replace(' ', '').replace('.', '')  # limpiar símbolos
    try:
        return int(s)
    except:
        return None

df['salary'] = df['salary'].apply(limpiar_salario)

In [ ]:
# Verificar resultado
print(df["salary"])

0              NaN
1              NaN
2        4000000.0
3        2000000.0
4        2500000.0
5        3200000.0
6        3500000.0
7              NaN
8        1423500.0
9        1800000.0
10             NaN
11             NaN
12       4000000.0
13       2999999.0
14       2847500.0
15       2000000.0
16       3000000.0
17       2500000.0
18       1800000.0
19             NaN
20             NaN
21       3000000.0
22       3000000.0
23       2000000.0
24       1800000.0
25       2200000.0
26       4318138.0
27       2500000.0
28       2250000.0
29             NaN
30       2500000.0
31       4000000.0
32       2500000.0
33       2912000.0
34       2499998.0
35       1927897.0
36       3105000.0
37       3067155.0
38       2630000.0
39             NaN
40       2499998.0
41             NaN
42             NaN
43             NaN
44       2200000.0
45             NaN
46       2712574.0
47       4100000.0
48             NaN
49       1500000.0
50       1820600.0
51             NaN
52          

In [10]:
# 2. Función para extraer el primer valor de 7 cifras después de $ o palabra SALARIO
def extract_main_salary(text):
    if pd.isna(text):
        return None

    text = text.lower()

    # Buscar después de "$" o "salario"
    match = re.search(r"(?:\$|salario[:\s]*)\s*([\d.,]{7,})", text)
    if match:
        value = match.group(1)
        try:
            # Convertir a número (quita puntos miles y usa coma decimal si aplica)
            return float(value.replace(".", "").replace(",", "."))
        except ValueError:
            return None
    return None

# 3. Guardar la columna original de salary
df["salary_original"] = df["salary"]

# 4. Aplicar la extracción
df["salary_extracted"] = df["description"].apply(extract_main_salary)

# 5. Rellenar solo los NaN originales en 'salary'
df["salary"] = df["salary_original"].combine_first(df["salary_extracted"])

df.drop(columns=["salary_original", "salary_extracted"], inplace=True)


In [ ]:
df['salary'] = df['salary'].astype('Int64')  # si ya está limpio

In [12]:
conteo_ciudades = df['salary'].value_counts().sort_values(ascending=False).reset_index()
conteo_ciudades.columns = ['salary', 'frecuencia']
pd.set_option('display.max_rows', None)  # Muestra todas las filas
print(conteo_ciudades)

             salary  frecuencia
0           1423500         176
1           2000000          89
2           1800000          69
3           2500000          60
4           1500000          56
5           3000000          51
6           3500000          27
7           4000000          26
8           1600000          22
9           2200000          22
10          2300000          18
11          1700000          18
12          1300000          17
13          2600000          16
14          1900000          15
15          2800000          15
16          1935000          13
17          1423000          11
18          3274000          10
19          2100000          10
20          2700000           9
21          2540000           9
22          3200000           9
23          5000000           9
24          3300000           9
25          2400000           8
26          2123500           7
27          4300000           6
28          4500000           6
29          1623500           5
30      

Editar valores específicos manualmente, que tienen errores poco comunes para no perder información

In [13]:
df.loc[df["salary"] == 20000002500000, "salary"] = 2500000
df.loc[df["salary"] ==  230000000, "salary"] = 2300000
df.loc[df["salary"] ==  16230000, "salary"] = 1623000
df.loc[df["salary"] ==  13000000, "salary"] = 1300000
df.loc[df["salary"] ==  700000, "salary"] = 1423500 
df.loc[df["salary"] ==  400000, "salary"] = 4000000 
df.loc[df["salary"] ==  300000, "salary"] = 3000000 
df.loc[df["salary"] ==  245000, "salary"] = 2450000
df.loc[df["salary"] ==  295000, "salary"] = 2950000
df.loc[df["salary"] ==  500000, "salary"] = 1423500
df.loc[df["salary"] ==  1600000400000, "salary"] = 1600000
df.loc[df["salary"] ==  999999, "salary"] = np.nan
df.loc[df["salary"] ==  610160, "salary"] =  1423500
df.loc[df["salary"] ==   17635000, "salary"] =   1763500
df.loc[df["salary"] ==   10000000, "salary"] =   1423500

## Clean age

In [15]:
df["age"] = df["age"].str.strip()  # Eliminar espacios en blanco
df["age"] = df["age"].replace("No disponible", "A partir de 18 años")


## Clean Contract

In [16]:
df["contract"] = df["contract"].str.replace("Contrato ", "", regex=False).str.strip()

In [17]:
df.head(5)

,url,title,company,rating,salary,contract,schedule,description,education,experience,age,skills,city,department
0,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de programación y datos - Medellín,Empleamos Temporales SAS,4.45,<NA>,de Obra o labor,Tiempo Completo,buscar talento programación análisis datosempl...,Universidad / Carrera tecnológica,1 año de experiencia,A partir de 18 años,No disponible,Medellín,Antioquia
1,https://co.computrabajo.com/ofertas-de-trabajo...,Analista Operaciones / Analista de datos,Corporación Interactuar,NaN,<NA>,a término indefinido,Tiempo Completo,"apasionado análisis dato , oferta corporación ...",Universidad / Carrera Profesional,No disponible,A partir de 18 años,No disponible,Bello,Antioquia
2,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos / Analista BI // Manejo de i...,BRM S.A.S,4.24,4000000,a término indefinido,Tiempo Completo,"detective dato nato , capaz encontrar aguja co...",Universidad / Carrera Profesional,2 años de experiencia,A partir de 18 años,No disponible,Medellín,Antioquia
3,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos,Alianza Temporal S.A.S,4.31,2000000,de Obra o labor,Tiempo Completo,importante empresa sector logistico transporte...,Universidad / Carrera Profesional,3 años de experiencia,A partir de 18 años,No disponible,"Bogotá, D.C.","Bogotá, D.C."
4,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos / Experiencia en facturación...,BRM S.A.S,4.24,2500000,a término indefinido,Tiempo Completo,gurú dato financiero obsesión exactitud factur...,Universidad / Carrera técnica,1 año de experiencia,A partir de 18 años,No disponible,"Bogotá, D.C.","Bogotá, D.C."


## Clean experience


In [18]:
import pandas as pd
import re

def procesar_experiencia(df):
    def extract_experience_from_description(row):
        text = str(row['description']).lower()

        # Detectar si dice que NO requiere experiencia
        patrones_sin_experiencia = [
            r"sin\s+experiencia",
            r"no\s+requiere\s+experiencia",
            r"no\s+se\s+necesita\s+experiencia",
            r"no\s+necesita[s]?\s+experiencia",
            r"experiencia\s+no\s+necesaria",
            r"no\s+es\s+necesario\s+experiencia",
            r"experiencia\s+no\s+requerida",
            r"no\s+es\s+indispensable\s+experiencia",
            r"experiencia\s+no\s+indispensable"
        ]
        if any(re.search(pat, text) for pat in patrones_sin_experiencia):
            return 0.0

        # Patrones para extraer experiencia explícita
        patterns = [
            r"(\d+)\s*(a|-|–|—)\s*(\d+)\s*(año|mes)",
            r"(\d+)\s*(to|–|—|\-| )\s*(\d+)\s*(year|month)",
            r"(\d+)\s*año\s+de\s+experiencia",
            r"mínimo\s+(de\s+)?(\d+)\s*año",
            r"experiencia\s+(de\s+|mínimo\s+de\s+)?(\d+)\s*año",
            r"al\s+menos\s+(\d+)\s*año",
            r"experiencia.*?(\d+)\s*año",
            r"(\d+)\s*mes\s+de\s+experiencia",
            r"mínimo\s+(de\s+)?(\d+)\s*mes",
            r"experiencia\s+(de\s+|mínimo\s+de\s+)?(\d+)\s*mes",
            r"al\s+menos\s+(\d+)\s*mes",
            r"experiencia.*?(\d+)\s*mes",
            r"(\d+)\s*year\s*(of)?\s*experience",
            r"at\s+least\s+(\d+)\s*year",
            r"minimum\s+of\s+(\d+)\s*year",
            r"(?:require[d]?[:\s]*)+(\d+)\s*year",
            r"(\d+)\s*month\s*(of)?\s*experience",
            r"at\s+least\s+(\d+)\s*month",
            r"minimum\s+of\s+(\d+)\s*month",
            r"(?:require[d]?[:\s]*)+(\d+)\s*month",

            # Casos como '3 año experiencia' o '2 año experiencia'
            r"(\d+)\s*año\s+experiencia",
            r"(\d+)\s*mes\s+experiencia",

            # Números escritos en palabras
            r"(uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciséis|diecisiete|dieciocho|diecinueve|veinte)\s+año\s+experiencia",
            r"(uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciséis|diecisiete|dieciocho|diecinueve|veinte)\s+mes\s+experiencia",

            # Casos unidos o mal escritos como 'c1experiencia'
            r"experiencia.*?(uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciséis|diecisiete|dieciocho|diecinueve|veinte|\d+)",

            # Casos con errores o formatos alternativos
            r"experiencio\s*:\s*indispensable\s*(\d+)\s*(mes|año)",
        ]

        patrones_genericos_experiencia = [
            r"\bexperiencia\s+proceso\s+(administrativo|logístico|contable|comercial)",
            r"\brequerir\s+experiencia\s+(previo|previa)",
            r"\bexperiencia\s+(previo|previa|necesaria|requerida|mínima)",
        ]

        numeros_palabras = {
            'uno': 1, 'una': 1, 'dos': 2, 'tres': 3, 'cuatro': 4, 'cinco': 5,
            'seis': 6, 'siete': 7, 'ocho': 8, 'nueve': 9, 'diez': 10,
            'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quince': 15,
            'dieciséis': 16, 'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19, 'veinte': 20
        }

        valores_encontrados = []

        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                for g in match.groups():
                    if not g:
                        continue
                    g = g.strip()
                    if g.isdigit():
                        num = int(g)
                    elif g in numeros_palabras:
                        num = numeros_palabras[g]
                    else:
                        continue
                    if "mes" in match.group(0) or "month" in match.group(0):
                        valores_encontrados.append(round(num / 12, 2))
                    else:
                        valores_encontrados.append(float(num))

        if valores_encontrados:
            return min(valores_encontrados)

        if any(re.search(pat, text) for pat in patrones_genericos_experiencia):
            return 1.0

        return None

    def clean_experience_value(exp):
        if pd.isna(exp):
            return None
        exp = str(exp).lower().strip()
        match = re.search(r"(\d+)", exp)
        if not match:
            return None
        value = int(match.group(1))
        if "mes" in exp or "month" in exp:
            return round(value / 12, 2)
        else:
            return float(value)

    # Limpieza inicial
    df["experience"] = df["experience"].apply(clean_experience_value)

    # Extraer experiencia desde descripción
    df["experience_extracted"] = df.apply(extract_experience_from_description, axis=1)

    # Rellenar experiencia nula con extraída
    df["experience"] = df["experience"].combine_first(df["experience_extracted"])

    # Contrato de aprendizaje = 0.0 si no tiene valor aún
    df.loc[
        (df["experience"].isna()) &
        (df["contract"].str.lower().str.contains("aprendizaje", na=False)),
        "experience"
    ] = 0.0

    # Rellenar con moda si sigue faltando
    if df["experience"].isna().any():
        moda = df["experience"].mode()
        if not moda.empty:
            df["experience"].fillna(moda[0], inplace=True)

    # Eliminar columnas auxiliares
    df.drop(columns=["experience_extracted"], inplace=True)

    return df


In [19]:
df = procesar_experiencia(df)

C:\Users\John\AppData\Local\Temp\ipykernel_9112\3899925781.py:135: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["experience"].fillna(moda[0], inplace=True)


## Crear categorias

In [20]:
# Cargar modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Concatenar todos los títulos
text = " ".join(str(title) for title in df["title"]).lower()

# Limpiar caracteres especiales
text = re.sub(r"[^\w\sñáéíóúü]", "", text)

# Procesar texto con spaCy
doc = nlp(text)

# Filtrar lemas que no sean stopwords ni signos de puntuación
filtered_lemmas = [
    token.lemma_ for token in doc
    if not token.is_stop and not token.is_punct and token.lemma_.strip()
]

# Contar frecuencia
word_counts = Counter(filtered_lemmas)

# Mostrar las 20 palabras más frecuentes
top_20 = word_counts.most_common(50)
for word, freq in top_20:
    print(f"{word}: {freq}")


analista: 986
comercial: 433
asesor: 403
dato: 162
venta: 129
experiencia: 123
bogotá: 100
asesora: 98
center: 97
sector: 88
call: 86
servicio: 84
trabajo: 73
cliente: 71
casa: 66
ref: 64
financiero: 61
externo: 60
senior: 51
sistema: 50
contable: 45
moto: 43
operación: 41
administrativo: 40
cali: 40
junior: 39
gestión: 36
información: 35
auxiliar: 35
analyst: 34
datar: 31
inmediato: 31
telecomunicación: 30
tat: 30
año: 29
salud: 29
costo: 29
bi: 28
soporte: 28
base: 27
bogota: 27
compra: 27
urgente: 26
seguridad: 25
calidad: 25
bilingüe: 24
mercadeo: 24
contratación: 24
digital: 24
medellín: 22


In [21]:
# Función para asignar múltiples categorías a un título
def asignar_multiples_categorias(title):
    title_lower = title.lower()
    categorias_detectadas = []

    if "contable" in title_lower:
        categorias_detectadas.append("contable")
    if "dato" in title_lower or "informacion" in title_lower or "data" in title_lower:
        categorias_detectadas.append("datos")
    if "analista" in title_lower or "bi" in title_lower or "analyst" in title_lower:
        categorias_detectadas.append("analista")
    if "administrativo" in title_lower or "financiero" in title_lower or "business" in title_lower:
        categorias_detectadas.append("financiero")
    if "asesor" in title_lower or "asesora" in title_lower or "cliente" in title_lower or "venta" in title_lower or "servicio" in title_lower or "comercial" in title_lower or "compra" in title_lower:
        categorias_detectadas.append("asesor")
    if "especialista" in title_lower or "desarrollador" in title_lower or "programador" in title_lower or "software" in title_lower or "ingeniero" in title_lower or "senior" in title_lower or "bilingüe" in title_lower:
        categorias_detectadas.append("especialista")
    if "consultor" in title_lower or "soporte" in title_lower or "externo" in title_lower:
        categorias_detectadas.append("consultor")
    if "call" in title_lower or "center" in title_lower or "telecomunicacion" in title_lower:
        categorias_detectadas.append("call center")
    if "auxiliar" in title_lower or "junior" in title_lower or "jr" in title_lower or "practicante" in title_lower:
        categorias_detectadas.append("auxiliar")
    if "lider" in title_lower or "jefe" in title_lower or "supervisor" in title_lower or "responsable" in title_lower or "gerente" in title_lower:
        categorias_detectadas.append("lider")

    return ", ".join(set(categorias_detectadas)) if categorias_detectadas else "otros"

# Aplicar al DataFrame
df["category"] = df["title"].apply(asignar_multiples_categorias)

In [22]:
# Filtra las filas que quedaron como "otros"
otros_titles = df[df["category"] == "otros"]["title"].unique()
print(otros_titles)

['Conductor/a licencia c2 para planta de residuos peligrosos / Mosquera - Mosquera'
 'Convocatoria Jueves de Empleo Cali'
 'Back Office  / Manejo de inglés B2 o C1 - Dominio de Excel intermedio avanzado'
 'Psicóloga/o de  selección Facatativá con experiencia sector floricultor'
 'Coordinador de Costos - Sector Transporte y Logística'
 'Auditor Control Interno - Medellin' 'Inspector de calidad - Yumbo'
 'Coordinador de sistemas - Tiempo completo'
 'Líder de pauta digital - Trafficker Digital'
 'Tecnólogo en Sistemas - Tecnólogo en Sistemas'
 'Técnico en Contratación en Salud'
 'Documentador/a de Procesos - Ing. Industrial o de procesos'
 'Profesional Talento Humano People Analytics - People Analytics'
 'Se busca personal área Reporting. 6to Semestre en Ingeniería de sistemas - Economía o a fines ¡No dejes pasar la oportunidad!'
 'Ejecutivo de cuentas Inhause'
 'Técnico en Sistemas con Enfoque en Plataforma MiPres'
 '(Aprendiz) Técnico o Tecnólogo en Talento Humano - Gestión Administrati

## Clean Skills

In [23]:
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_punct and not token.is_space])


In [24]:
# Filtrar nulos y valores no disponibles
skills_series = df['skills'].dropna()
skills_series = skills_series[~skills_series.str.lower().str.contains("no disponible")]

# Separar por coma, limpiar espacios y convertir a minúsculas
skill_list = skills_series.str.split(',').explode().str.strip().str.lower()

# Lematizar cada habilidad
skill_list_lemmatized = skill_list[skill_list != ""].apply(lemmatize_text)

# Quitar duplicados y ordenar
unique_skills = sorted(skill_list_lemmatized.dropna().unique())

# Mostrar resultados
print(f"Se encontraron {len(unique_skills)} habilidades únicas lematizadas:")
for skill in unique_skills:
    print(f"- {skill}")


Se encontraron 140 habilidades únicas lematizadas:
- aceptación de error y fracaso
- adaptación al cambio
- administración
- administración de archivo
- administración de sistema
- adobe after effects
- adobe ilustrator
- adobe indesign
- adobe photoshop
- adobe premier
- agile
- análisis
- análisis de coste
- análisis financiero
- aprendizaje
- asesoer comercial
- asp.net
- asp.net mvc
- atención al cliente
- auditoria
- auditoria interno
- autocad
- autoconfianza
- big datar
- budget
- c
- calidad
- call center
- capacidad de decisión
- cloud computing
- comercio exterior
- compra
- comunicación y persuasión
- consultoer
- contabilidad
- continuous deployment
- coreldraw
- creación de equipo
- creatividad
- crm
- css
- desarrollo de negocio
- desarrollo de producto
- digitalización de documento
- dirección
- dirección de venta
- diseño
- django
- electrónico
- enseñanza
- erp
- facturación
- finanza
- gestión
- gestión de coste
- gestión de el emoción
- gestión de equipo
- gestión de

In [25]:
gestion_planeacion = {
    'gestion', 'planificacion', 'proyecto', 'mejora', 'reporte', 'presupuesto', 'tiempo', 'equipo', 'cambio'
}

finanzas_contabilidad = {
    'finanza', 'contabilidad', 'registro contable', 'liquidacion', 'facturacion', 'conciliacion bancaria', 'costo'
}

auditoria_calidad = {
    'auditoria', 'auditoria interno', 'iso 9001', 'lean', 'lean manufacturing', 'lean six sigma'
}

recursos_humanos = {
    'recurso humano', 'psicologia', 'reclutamiento', 'personal'
}

logistica_operaciones = {
    'inventario', 'base dato', 'analisis', 'contabilizar'
}

desarrollo_tecnologia_empresarial = {
    'continuous deployment', 'test driven development'
}

lenguajes_bases_datos = {
    'python', 'r', 'sql', 'sql server', 'mysql', 'postgresql', 'oracle'
}

bi_visualizacion = {
    'power bi', 'pbi'
}

sistemas_operativos = {
    'linux', 'windows', 'unix'
}

microsoft_office = {
    'excel', 'word', 'powerpoint', 'access', 'outlook', 'office', 'project'
}

erp_crm = {
    'erp', 'crm'
}

gis_cad_diseno = {
    'arcgis', 'autocad', 'coreldraw'
}

web_programacion = {
    'javascript', 'jquery', 'php', 'wordpress'
}

adobe_suite = {
    'photoshop', 'illustrator', 'indesign', 'after effects', 'premiere'
}

otros_tech = {
    'openoffice', 'tecnologia', 'servidor', 'digitalizacion', 'software contabilidad'
}

big_data_procesamiento = {
    'elastic search', 'big data', 'spark', 'hadoop', 'impala', 'pentaho'
}

redes = {
    'red', 'red social', 'networking'
}

personales = {
    'adaptacion', 'autoconfianza', 'iniciativa', 'logro', 'organizacion', 'aprendizaje', 'creatividad',
    'innovacion', 'aceptacion error', 'ensenar'
}

interpersonales = {
    'comunicacion', 'persuasion', 'negociacion', 'conflicto', 'problema', 'presentacion', 'liderazgo', 'trabajo equipo'
}

comerciales_servicio = {
    'habilidad comercial', 'asesor comercial', 'consultor', 'soporte cliente', 'servicio cliente', 'atencion cliente',
    'atencion al cliente', 'direccion venta', 'comercio exterior', 'venta', 'mercadeo', 'marketing', 'relacion publica'
}

sectores_especificos = {
    'construccion', 'retail', 'inmobiliario', 'calidad', 'manipulacion alimento', 'salud', 'salud ocupacional'
}

otros_operativos = {
    'archivo', 'sistema', 'gestion', 'administracion', 'logistica', 'transporte', 'inspeccion'
}

idiomas = {
    "espanol", "ingles", "frances", "portugues", "aleman", "chino", "japones", "coreano",
    "ruso", "arabe", "hindi", "bengali", "italiano", "turco", "polaco", "neerlandes",
    "sueco", "noruego", "finlandes", "danes", "checo", "griego", "ucraniano", "hungaro",
    "tailandes", "vietnamita", "indonesio", "malayo"
}

In [26]:
categorias_skills = {
    'gestion_planeacion': gestion_planeacion,
    'finanzas_contabilidad': finanzas_contabilidad,
    'auditoria_calidad': auditoria_calidad,
    'recursos_humanos': recursos_humanos,
    'logistica_operaciones': logistica_operaciones,
    'desarrollo_tecnologia_empresarial': desarrollo_tecnologia_empresarial,
    'lenguajes_bases_datos': lenguajes_bases_datos,
    'bi_visualizacion': bi_visualizacion,
    'sistemas_operativos': sistemas_operativos,
    'microsoft_office': microsoft_office,
    'erp_crm': erp_crm,
    'gis_cad_diseno': gis_cad_diseno,
    'web_programacion': web_programacion,
    'adobe_suite': adobe_suite,
    'otros_tech': otros_tech,
    'big_data_procesamiento': big_data_procesamiento,
    'redes': redes,
    'personales': personales,
    'interpersonales': interpersonales,
    'comerciales_servicio': comerciales_servicio,
    'sectores_especificos': sectores_especificos,
    'otros_operativos': otros_operativos,
    'idiomas': idiomas,
}


In [ ]:
import pandas as pd
import re
import spacy
import unicodedata

# Cargar modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

# Normalización y lematización
def normalizar_lemmatizar(texto):
    if pd.isna(texto):
        return ""
    texto = str(texto).lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')  # quitar tildes
    texto = re.sub(r"[^a-z0-9\s]", " ", texto)  # quitar signos de puntuación
    doc = nlp(texto)
    lemas = [token.lemma_ for token in doc if not token.is_stop]
    return " ".join(lemas)

# Aplicar lematización
df["description_lem"] = df["description"].apply(normalizar_lemmatizar)

# Palabras cortas que requieren búsqueda exacta
PALABRAS_CORTAS = {'r', 'sap', 'sql', 'erp', 'crm', 'php', 'arcgis'}

# Diccionario de categorías actualizado
categorias_skills = {
    'gestion_planeacion': gestion_planeacion,
    'finanzas_contabilidad': finanzas_contabilidad,
    'auditoria_calidad': auditoria_calidad,
    'recursos_humanos': recursos_humanos,
    'logistica_operaciones': logistica_operaciones,
    'desarrollo_tecnologia_empresarial': desarrollo_tecnologia_empresarial,
    'lenguajes_bases_datos': lenguajes_bases_datos,
    'bi_visualizacion': bi_visualizacion,
    'sistemas_operativos': sistemas_operativos,
    'microsoft_office': microsoft_office,
    'erp_crm': erp_crm,
    'gis_cad_diseno': gis_cad_diseno,
    'web_programacion': web_programacion,
    'adobe_suite': adobe_suite,
    'otros_tech': otros_tech,
    'big_data_procesamiento': big_data_procesamiento,
    'redes': redes,
    'personales': personales,
    'interpersonales': interpersonales,
    'comerciales_servicio': comerciales_servicio,
    'sectores_especificos': sectores_especificos,
    'otros_operativos': otros_operativos,
    'idiomas': idiomas,
}

# Función para detectar si alguna palabra clave está en el texto
def detectar_skills(texto, conjunto_skills):
    for skill in conjunto_skills:
        if skill in PALABRAS_CORTAS:
            if re.search(rf'\b{re.escape(skill)}\b', texto):
                return 1
        elif skill in texto:
            return 1
    return 0

# Crear columnas binarias para cada categoría
for categoria, conjunto in categorias_skills.items():
    df[categoria] = df["description_lem"].apply(lambda texto: detectar_skills(texto, conjunto))


In [28]:
# Total de valores nulos por columna
df.isna().sum()


url                                    0
title                                  0
company                                0
rating                               698
salary                               408
contract                               0
schedule                               0
description                            0
education                              0
experience                             0
age                                    0
skills                                 0
city                                   0
department                             0
category                               0
description_lem                        0
gestion_planeacion                     0
finanzas_contabilidad                  0
auditoria_calidad                      0
recursos_humanos                       0
logistica_operaciones                  0
desarrollo_tecnologia_empresarial      0
lenguajes_bases_datos                  0
bi_visualizacion                       0
sistemas_operati

In [30]:
df.head(5)

,url,title,company,rating,salary,contract,schedule,description,education,experience,...,adobe_suite,otros_tech,big_data_procesamiento,redes,personales,interpersonales,comerciales_servicio,sectores_especificos,otros_operativos,idiomas
0,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de programación y datos - Medellín,Empleamos Temporales SAS,4.45,<NA>,de Obra o labor,Tiempo Completo,buscar talento programación análisis datosempl...,Universidad / Carrera tecnológica,1.0,...,0,0,0,0,0,0,0,0,0,0
1,https://co.computrabajo.com/ofertas-de-trabajo...,Analista Operaciones / Analista de datos,Corporación Interactuar,NaN,<NA>,a término indefinido,Tiempo Completo,"apasionado análisis dato , oferta corporación ...",Universidad / Carrera Profesional,1.0,...,0,0,0,0,1,0,0,0,1,0
2,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos / Analista BI // Manejo de i...,BRM S.A.S,4.24,4000000,a término indefinido,Tiempo Completo,"detective dato nato , capaz encontrar aguja co...",Universidad / Carrera Profesional,2.0,...,0,0,0,0,0,1,0,0,0,0
3,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos,Alianza Temporal S.A.S,4.31,2000000,de Obra o labor,Tiempo Completo,importante empresa sector logistico transporte...,Universidad / Carrera Profesional,3.0,...,0,0,0,0,0,0,0,0,1,0
4,https://co.computrabajo.com/ofertas-de-trabajo...,Analista de datos / Experiencia en facturación...,BRM S.A.S,4.24,2500000,a término indefinido,Tiempo Completo,gurú dato financiero obsesión exactitud factur...,Universidad / Carrera técnica,1.0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
import os

# Ruta relativa desde el notebook
output_path = os.path.join("..", "data", "clean_offers.csv")
df.to_csv(output_path, index=False, encoding='utf-8')
